# Random Forest Classifier

In [1]:
import   numpy  as   np 
import   pandas  as   pd 
from  sklearn . metrics  import   accuracy_score 
from  sklearn  import   tree 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut

In [42]:
dataset = pd.read_csv('C:\\Users\\Fabiel Fernando\\Desktop\\PROVA\\classificacao_Q4.csv')
X = dataset.iloc[:, 0:100].values
#y = np.array(dataset['target'])
y = dataset['target'].astype('category')

In [43]:
# Transformando os dados em categóricos
#from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#labelencoder_y = LabelEncoder()
#y = labelencoder_y.fit_transform(y)

In [44]:
y.dtype

CategoricalDtype(categories=[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0], ordered=False)

### Porcentagem da variável resposta

In [7]:
resposta = dataset['target']
count = pd.DataFrame(resposta.value_counts())
percent = pd.DataFrame(resposta.value_counts(normalize = True)*100)
table = pd.concat([count, percent], axis = 1)
table.columns = ['# target', '% target']
table

,# target,% target
2.0,153,10.200000
8.0,153,10.200000
3.0,152,10.133333
0.0,150,10.000000
4.0,150,10.000000
1.0,149,9.933333
6.0,149,9.933333
9.0,149,9.933333
5.0,148,9.866667
7.0,147,9.800000


#### Treino e Teste

In [8]:
X_train ,  X_test ,  y_train ,  y_test  =  train_test_split (  X ,  y ,  test_size  =  0.3 ,  random_state  =  100 ) 

### Classificador Floresta aleatória

In [9]:
#criterio = "gini"
criterio = "entropy"

In [10]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(criterion = criterio,
                                       max_depth=3,
                                       min_samples_leaf = 5, 
                                       min_samples_split = 10,   
                                       n_estimators=100, 
                                       max_features='auto', 
                                       oob_score=True, 
                                       random_state=42, 
                                       n_jobs=-1)
classifier.fit(X_train, y_train)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

### Precisão do classificador 

In [11]:
pred_test = classifier.predict(X_test)
pred_train = classifier.predict(X_train)

y_true = np.array(y_test.copy())
y_pred = np.array(pred_test.copy())

y_true_treino = np.array(y_train.copy())
y_pred_treino = np.array(pred_train.copy())

#### Tabela com cálculo de vária métricas conjunto treino

In [12]:
from sklearn.metrics import classification_report
print(classification_report(y_true_treino, y_pred_treino), 
    print ("Acurácia para o treino é ", accuracy_score(y_true_treino,y_pred_treino)))

Acurácia para o treino é  0.6514285714285715
             precision    recall  f1-score   support

        0.0       0.66      0.88      0.75       110
        1.0       0.54      0.59      0.56       102
        2.0       0.83      0.41      0.55       109
        3.0       0.68      0.72      0.70       105
        4.0       0.56      0.46      0.51       107
        5.0       0.74      0.52      0.61       100
        6.0       0.60      0.81      0.69       109
        7.0       0.74      0.59      0.66       101
        8.0       0.68      0.70      0.69        99
        9.0       0.62      0.81      0.71       108

avg / total       0.67      0.65      0.64      1050
 None


#### Tabela com cálculo de vária métricas conjunto teste

In [13]:
print(classification_report(y_true, y_pred), 
    print ("Acurácia para o treino é ", accuracy_score(y_true,y_pred)))

Acurácia para o treino é  0.5555555555555556
             precision    recall  f1-score   support

        0.0       0.52      0.88      0.65        40
        1.0       0.58      0.62      0.60        47
        2.0       0.61      0.32      0.42        44
        3.0       0.64      0.62      0.63        47
        4.0       0.35      0.33      0.34        43
        5.0       0.69      0.56      0.62        48
        6.0       0.41      0.55      0.47        40
        7.0       0.61      0.48      0.54        46
        8.0       0.62      0.52      0.57        54
        9.0       0.59      0.73      0.65        41

avg / total       0.57      0.56      0.55       450
 None


# Ajustando o classificador com Grid Search

In [14]:
from sklearn.model_selection import KFold
cv_kfold = KFold(10, shuffle = False)
param_grid = { 
    'n_estimators': [2, 5],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5],
    'criterion' :['gini', 'entropy']
}

In [15]:
clf_rfc_cv = GridSearchCV(RandomForestClassifier(), param_grid, cv = cv_kfold)

In [16]:
clf_rfc_cv.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [2, 5], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [4, 5], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [17]:
model = clf_rfc_cv.best_estimator_
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Métricas

In [18]:
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV
predicted = cross_val_score(model, X, y, cv = cv_kfold, scoring = 'accuracy')

In [19]:
print ("Acurácia média é", predicted.mean())

Acurácia média é 0.6046666666666667


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        0.0       0.52      0.88      0.65        40
        1.0       0.58      0.62      0.60        47
        2.0       0.61      0.32      0.42        44
        3.0       0.64      0.62      0.63        47
        4.0       0.35      0.33      0.34        43
        5.0       0.69      0.56      0.62        48
        6.0       0.41      0.55      0.47        40
        7.0       0.61      0.48      0.54        46
        8.0       0.62      0.52      0.57        54
        9.0       0.59      0.73      0.65        41

avg / total       0.57      0.56      0.55       450



In [21]:
#print(confusion_matrix(y_test, y_pred))

# Validação Cruzada

#### k-fold

In [22]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10, random_state=42)
model = RandomForestClassifier()
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
results.mean(), results.std()

(0.6693333333333333, 0.04954907780283213)

#### LOOCV

In [23]:
from sklearn.model_selection import LeaveOneOut

In [24]:
model = RandomForestClassifier()
accuracies = cross_val_score(model, X=X_train, y=y_train, cv=LeaveOneOut())
accuracies.mean()

0.6571428571428571

#### Repeat CV

In [25]:
from sklearn.model_selection import RepeatedKFold

In [26]:
cv_repeat = RepeatedKFold(n_splits=6, n_repeats=3, random_state=42)
model = RandomForestClassifier()
accuracies = cross_val_score(model, X=X_train, y=y_train, cv=cv_repeat)
accuracies.mean()

0.6276190476190475

#### Separando as k primeiras observações para treino e o restante para teste

In [27]:
X_treino = dataset.iloc[0:499, 0:99].values
y_treino = dataset.iloc[0:499, 100].values


X_teste = dataset.iloc[500:1500, 0:99].values
y_teste = dataset.iloc[500:1500, 100].values

In [28]:
#criterio = 'gini'
criterio = 'entropy'

In [29]:
clf = RandomForestClassifier(criterion = criterio,
                                       max_depth=3,
                                       min_samples_leaf = 1, 
                                       min_samples_split = 10,   
                                       n_estimators=100, 
                                       max_features='auto', 
                                       oob_score=True, 
                                       random_state=42, 
                                       n_jobs=-1)
clf.fit(X_treino, y_treino)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=3, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)

#### Precisão do classificador da Árvore de Decisão

In [30]:
pred_teste = clf.predict(X_teste)
pred_treino = clf.predict(X_treino)

y_true = np.array(y_teste.copy())
y_pred = np.array(pred_teste.copy())

y_true_treino = np.array(y_treino.copy())
y_pred_treino = np.array(pred_treino.copy())

#### Tabela com cálculo de vária métricas conjunto treino

In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_true_treino, y_pred_treino), 
    print ("Acurácia para o treino é ", accuracy_score(y_true_treino,y_pred_treino)))

Acurácia para o treino é  0.687374749498998
             precision    recall  f1-score   support

        0.0       0.59      0.85      0.70        55
        1.0       0.65      0.77      0.70        48
        2.0       0.89      0.38      0.53        45
        3.0       0.68      0.81      0.74        58
        4.0       0.81      0.44      0.57        48
        5.0       0.83      0.54      0.66        46
        6.0       0.60      0.79      0.68        47
        7.0       0.74      0.67      0.70        51
        8.0       0.90      0.63      0.74        41
        9.0       0.64      0.87      0.74        60

avg / total       0.72      0.69      0.68       499
 None


#### Tabela com cálculo de vária métricas conjunto teste

In [32]:
print(classification_report(y_true, y_pred), 
    print ("Acurácia para o treino é ", accuracy_score(y_true,y_pred)))

Acurácia para o treino é  0.57
             precision    recall  f1-score   support

        0.0       0.55      0.80      0.65        95
        1.0       0.56      0.58      0.57       101
        2.0       0.65      0.29      0.40       108
        3.0       0.56      0.76      0.64        94
        4.0       0.63      0.24      0.34       102
        5.0       0.68      0.42      0.52       102
        6.0       0.55      0.75      0.64       102
        7.0       0.56      0.52      0.54        96
        8.0       0.73      0.55      0.63       111
        9.0       0.46      0.88      0.60        89

avg / total       0.60      0.57      0.55      1000
 None
